In [1]:
!pip install regex

In [2]:
from transformers import BartTokenizerFast, PreTrainedTokenizerFast
from tokenizers import SentencePieceBPETokenizer
import os
import json
import regex as re
import polars as pl

In [3]:
from transformers import logging
logging.set_verbosity_error() 

In [4]:
df = pl.read_parquet('/kaggle/input/nepbart-llm-dataset/nepbart_dataset/nepbart_train.parquet')

In [5]:
# text = """ hi my n"ame 'is।  she asked, 'what do you do}{} sir?????'  "what।" 'again?'"""
# # pattern = r'''(?<=[।?])["]?['](?=\s|$)''' chatgpt
# pattern = r'''(?<=[।?]['"]?)\s+''' myone
# pattern = r'[^?।]+[?।][\'"]?' claude findall one
# text = re.split(pattern, text)
# # text = [text[i] + text[i + 1] for i in range(0, len(text) - 1, 2)]
# print(len(text))
# text

In [6]:
# train_text = []
# for row in df.iter_rows(named=True):
#     # pattern = r'(?<=[।?])["\']?(?=\s|$)'
#     # pattern = r'[^?।]+[?।][\'"]?'
#     pattern = r'''(?<=[।?]['"]?)\s+'''
#     sentences = re.split(pattern, row['text'])

#     for sentence in sentences:
#         if sentence.strip() != '':
#             train_text.append(sentence.strip())

In [7]:
special_tokens = ["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
spm_tokenizer = SentencePieceBPETokenizer()
spm_tokenizer.train_from_iterator(
    df['text'].to_list(),
    # [f'/kaggle/working/vocab_data/{file}' for file in os.listdir('/kaggle/working/vocab_data')],
    vocab_size=32_768,
    min_frequency=10,
    limit_alphabet = 2_000,
    show_progress=True,
    special_tokens=special_tokens
)
# spm_tokenizer.save_model('./tokenizer')

In [8]:
del df

In [9]:
def configure_tokenizer(tokenizer, special_tokens, save_location):
    tokenizer_fast = PreTrainedTokenizerFast(tokenizer_object=tokenizer, special_tokens=special_tokens, clean_up_tokenization_spaces=True)
    tokenizer_fast.bos_token, tokenizer_fast.bos_token_id = "<s>", tokenizer.token_to_id("<s>")
    tokenizer_fast.pad_token, tokenizer_fast.pad_token_id = "<pad>", tokenizer.token_to_id("<pad>")
    tokenizer_fast.eos_token, tokenizer_fast.eos_token_id = "</s>", tokenizer.token_to_id("</s>")
    tokenizer_fast.unk_token, tokenizer_fast.unk_token_id = "<unk>", tokenizer.token_to_id("<unk>")
    tokenizer_fast.mask_token, tokenizer_fast.mask_token_id = "<mask>", tokenizer.token_to_id("<mask>")
    # and save for later!
    if not os.path.exists(save_location):
        os.mkdir(save_location)
    tokenizer_fast.save_pretrained(save_location)
    return tokenizer_fast

In [10]:
tokenizer_save_location = '/kaggle/working/tokenizer'

In [11]:
tokenizer = configure_tokenizer(spm_tokenizer, special_tokens, save_location=tokenizer_save_location)

In [12]:
tokenizer = BartTokenizerFast.from_pretrained(tokenizer_save_location)

In [13]:
# tokenizer_nepbart = BartTokenizerFast(
#     vocab_file="/kaggle/working/tokenizer/vocab.json", merges_file="/kaggle/working/tokenizer/merges.txt", clean_up_tokenization_spaces=True)

# configure_tokenizer(spm_tokenizer, special_tokens, save_location="/kaggle/working/tokenizer")
    
# # tokenizer.save_pretrained("/kaggle/working/tokenizer_nepbart")

In [14]:
# tokenizer("फ्लुको कारणले हुने पहिलोनेपाली भवकृष्ण भट्टराई", return_tensors='pt')

In [15]:
# tokenizer = AutoTokenizer.from_pretrained("tokenizer_nepbart")

In [16]:
tokenizer("फ्लुको कारणले हुने पहिलोनेपाली भवकृष्ण भट्टराई", return_tensors='pt', padding='max_length', max_length=10, truncation=True)

{'input_ids': tensor([[25436,   544,  3591,   743,  1132, 24704,  3867,  4191,  2371,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [17]:
tokenizer("\" बाढीको कारण जम्मा एक सय ८४ घर डुबानमा परेका छन्।", return_tensors='pt', padding='max_length', max_length=10, truncation=True)

{'input_ids': tensor([[ 1279, 14144,   960,  3166,   702,   903,  5629,  1068, 11881,  1830]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [25]:
tokenizer.tokenize(" ".join("फ्लुको, 'कारणले हुने पहिलोनेपाली भवकृष्ण भट्टराई'".split()))

['▁फ्लु',
 'को,',
 "▁'",
 'कारण',
 'ले',
 '▁हुने',
 '▁पहिलो',
 'नेपाली',
 '▁भव',
 'कृष्ण',
 '▁भट्टराई',
 "'"]

In [19]:
tokenizer.tokenize(" बाढीको कारण जम्मा एक सय ८४ घर डुबानमा परेका छन्।")

['▁बाढीको',
 '▁कारण',
 '▁जम्मा',
 '▁एक',
 '▁सय',
 '▁८४',
 '▁घर',
 '▁डुबानमा',
 '▁परेका',
 '▁छन्।']

In [20]:
tokenizer.tokenize("महायज्ञ गर्न दलित परिवारको उठीबास लगाएको घटनामा संवेदनशील भएर कार्य नगरेको भन्दै गृह मन्त्रालयले सिराहाका प्रमुख जिल्ला अधिकारी बासुदेव दाहाललाई मन्त्रालय तानेको छ।")

['▁महायज्ञ',
 '▁गर्न',
 '▁दलित',
 '▁परिवारको',
 '▁उठ',
 'ी',
 'बास',
 '▁लगाएको',
 '▁घटनामा',
 '▁संवेदनशील',
 '▁भएर',
 '▁कार्य',
 '▁नगरेको',
 '▁भन्दै',
 '▁गृह',
 '▁मन्त्रालयले',
 '▁सिराहा',
 'का',
 '▁प्रमुख',
 '▁जिल्ला',
 '▁अधिकारी',
 '▁बासुदेव',
 '▁दाहाललाई',
 '▁मन्त्रालय',
 '▁ता',
 'नेको',
 '▁छ।']

In [21]:
tokenizer("फ्लुको कारणले हुने पहिलोनेपाली भवकृष्ण भट्टराई", return_tensors='pt', padding='max_length', max_length=10, truncation=True)

{'input_ids': tensor([[25436,   544,  3591,   743,  1132, 24704,  3867,  4191,  2371,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [22]:
tokenizer.decode((tokenizer("फ्लुको कारणले हुने पहिलोनेपाली भवकृष्ण भट्टराई", padding='max_length', max_length=75, truncation=True)['input_ids']), skip_special_tokens=True)

'फ्लुको कारणले हुने पहिलोनेपाली भवकृष्ण भट्टराई'